In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# hydrogen.csv hydrogen_small.csv kick.csv Prepare_SQuAD_tiny.ipynb tfidf_features.csv tfidf_features_small.csv tsdm.csv

#Task 1

In [ ]:
df = pd.read_csv(r'C:\Users\stevi\OneDrive\Desktop\Assignment 2\Datasets\tfidf_features.csv')
hydrogen = pd.read_csv(r'C:\Users\stevi\OneDrive\Desktop\Assignment 2\Datasets\hydrogen.csv')
df.info

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(hydrogen['label'])

In [ ]:
# Prepare features
X = df
X

In [ ]:
from scipy.spatial.distance import pdist

X_mat = X.to_numpy()

dims = [10, 50, 100, 200, 400, 800, 1600, min(2500, X_mat.shape[1])]
mean_dists = []

for d in dims:
    cols = np.random.choice(X_mat.shape[1], size=d, replace=False)
    subset = X_mat[:, cols]
    dist = pdist(subset, metric='euclidean')
    mean_dists.append(np.mean(dist))

plt.plot(dims, mean_dists, marker='o')
plt.xlabel('Number of Dimensions')
plt.ylabel('Average Euclidean Distance')
plt.title('Curse of Dimensionality: Distance vs Dimensionality')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X.values)
X_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2","PC3", "PC4"])
X_pca

In [ ]:
X_pca.describe()

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
print("Total explained variance with 4 components:", sum(pca.explained_variance_ratio_))

In [ ]:
# Create a range of components to test (10 to 50)
comps_num = np.arange(1, 2500+1, 250)

comps_variance = []
for k in comps_num:
    pca = PCA(n_components=k, random_state=10)
    pca.fit(X.values)
    comps_variance.append(pca.explained_variance_ratio_.sum())

# Plot cumulative explained variance
plt.figure(figsize=(10,5))
plt.grid(True)
plt.plot(comps_num, comps_variance, marker='o')
plt.xlabel('# Components')
plt.ylabel('Total explained variance')
plt.title('PCA cumulative explained variance vs components')
plt.ylim(0, 1.0)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
# Create a range of components to test (10 to 50)
comps_num = np.arange(1, 300+1, 10)

comps_variance = []
for k in comps_num:
    pca = PCA(n_components=k, random_state=10)
    pca.fit(X.values)
    comps_variance.append(pca.explained_variance_ratio_.sum())

# Plot cumulative explained variance
plt.figure(figsize=(10,5))
plt.grid(True)
plt.plot(comps_num, comps_variance, marker='o')
plt.xlabel('# Components')
plt.ylabel('Total explained variance')
plt.title('PCA cumulative explained variance vs components')
plt.ylim(0, 1)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
# Create a range of components to test (10 to 50)
comps_num = np.arange(1, 25, 1)

comps_variance = []
for k in comps_num:
    pca = PCA(n_components=k, random_state=10)
    pca.fit(X.values)
    comps_variance.append(pca.explained_variance_ratio_.sum())

# Plot cumulative explained variance
plt.figure(figsize=(10,5))
plt.grid(True)
plt.plot(comps_num, comps_variance, marker='o')
plt.xlabel('# Components')
plt.ylabel('Total explained variance')
plt.title('PCA cumulative explained variance vs components')
plt.ylim(0, 0.175)
plt.show()

In [ ]:
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X.values)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
print("Total explained variance with 5 components:", sum(pca.explained_variance_ratio_))

In [ ]:
pca = PCA(n_components=2000)
X_pca = pca.fit_transform(X.values)

In [ ]:
# We can't use `plt.scatter()` directly as that only works for 2D, so first create a 3D subplot.
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection="3d")
# Now we can create the scatterplot.
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=y, s=20)

ax.legend(
    handles=scatter.legend_elements()[0],
    labels=["Not hydrogen-related", "Hydrogen-related"],
    title="PCA Tweet Relevance"
)
ax.set_title("PCA Visualisation of Hydrogen Tweet Dataset (3 Components)")
ax.set(xlabel="PC1", ylabel="PC2", zlabel="PC3")

plt.show()

In [ ]:
pca_2000 = PCA(n_components=2000, random_state=10)
X_pca2000 = pca_2000.fit_transform(X.values)

In [ ]:
from sklearn.manifold import TSNE
random_state = 10
tsne = TSNE(n_components=2, random_state=random_state)
tsne.fit(X_pca2000)

In [ ]:
print("KL divergence for 2 components:", tsne.kl_divergence_)

In [ ]:
# Create an array containing the numbers 1 through 3 inclusive, this will also be our X values.
tsne_comps_num = np.arange(1, 3+1)
tsne_comps_divergence = []
for num_components in tsne_comps_num:
  # Fit a PCA model to the specified number of components.
  tsne = TSNE(n_components=num_components, random_state=random_state)
  tsne.fit(X_pca2000)
  tsne_comps_divergence.append(tsne.kl_divergence_)
  print(f"KL Divergence for {num_components}: {tsne.kl_divergence_}")

In [ ]:
# Perplexity must range between 5 and 50. Use steps of 5, as computing with an interval of 1 would take too long.
tsne_perplexity_num = np.arange(5, 50+1, 5)
tsne_perplexity_divergence = []
for perplexity in tsne_perplexity_num:
  # Fit a TSNE model to the specified number of components.
  tsne = TSNE(n_components=2, perplexity=perplexity, random_state=random_state)
  tsne.fit(X_pca2000)
  tsne_perplexity_divergence.append(tsne.kl_divergence_)
  print(f"KL Divergence for perplexity of {perplexity}: {tsne.kl_divergence_}")

In [ ]:
# Add grid lines to improve visibility.
plt.figure(1, figsize=(12, 6))
plt.grid()
# Plot each of the data points, using a circular marker.
plt.plot(tsne_perplexity_num, tsne_perplexity_divergence, marker='o')
# Set the X axis to our component range, and the Y axis limits from 0 to 1 (as variance is a ratio between 0 and 1).
plt.xlabel("Perplexity")
plt.ylabel("KL Divergence")
plt.title("t-SNE Perplexity Tuning: KL Divergence vs Perplexity")
plt.show()

In [ ]:
tsne = TSNE(n_components=3, perplexity=5, random_state=random_state)
X_tsne = tsne.fit_transform(X_pca2000)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection="3d")
scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=y, s=20)
ax.legend(
    handles=scatter.legend_elements()[0],
    labels=["Not hydrogen-related", "Hydrogen-related"],
    title="t-SNE Tweet Relevance"
)
ax.set(xlabel="C1", ylabel="C2", zlabel="C3")
ax.set_title("t-SNE Visualisation of Hydrogen Tweet Dataset (3 Components)")
plt.show()

# Task 2

In [ ]:
# Function to inspect Qualitative variable with simple plot and basic statistics
def inspect_qualitative_variable(df, column):
    print(f"\n=== {column} ===")

    # Calculate basic statistics
    total = len(df)                                  # Total number of records
    missing = df[column].isnull().sum()              # Count of missing (NaN) values
    unique_vals = df[column].nunique(dropna=True)    # Number of unique categories (excluding NaN)

    # Print summary statistics
    print(f"Total values       : {total}")
    print(f"Missing values     : {missing}")
    print(f"Unique categories  : {unique_vals}")
    print("\nValue counts:")
    print(df[column].value_counts(dropna=False))     # Print count of each category including NaN

    # Plot count distribution as a bar chart
    plt.figure(figsize=(8, 5))                        # Set plot size
    sns.countplot(
        data=df,
        x=column,
        hue=column,                                   # Apply color separation by the same variable
        order=df[column].value_counts().index,        # Display bars in descending order of frequency
        palette='pastel'                              # Use a pastel color palette
    )
    plt.title(f'{column} Distribution')               # Set plot title
    plt.xlabel(column.capitalize())                   # Label x-axis with column name
    plt.ylabel('Count')                               # Label y-axis
    plt.xticks(rotation=45)                           # Rotate x-axis labels for readability
    plt.tight_layout()                                # Adjust layout to prevent overlap
    plt.show()                                        # Display the plot

In [ ]:
# Define a function to inspect a Quantitative variable in a DataFrame
def inspect_quantitative_variable(df, column):
    print(f"\n=== {column} ===")

    # Calculate basic statistics
    total = len(df)                                  # Total number of entries
    missing = df[column].isnull().sum()              # Count of missing values
    negative = (df[column] < 0).sum()                # Count of negative values
    zero = (df[column] == 0).sum()                   # Count of zero values
    min_val = df[column].min()                       # Minimum value
    max_val = df[column].max()                       # Maximum value
    mean = df[column].mean()                         # Mean of the column
    median = df[column].median()                     # Median of the column
    std = df[column].std()                           # Standard deviation

    # Print the calculated statistics
    print(f"Total values     : {total}")
    print(f"Missing values   : {missing}")
    print(f"Negative values  : {negative}")
    print(f"Zero values      : {zero}")
    print(f"Min              : {min_val}")
    print(f"Max              : {max_val}")
    print(f"Mean             : {mean:.2f}")
    print(f"Median           : {median:.2f}")
    print(f"Std. Deviation   : {std:.2f}")

    # Visualise the distribution of values using a histogram and boxplot
    plt.figure(figsize=(12, 4))                      # Set up a wide figure for side-by-side plots

    # Plot histogram with KDE (kernel density estimate)
    plt.subplot(1, 2, 1)
    sns.histplot(df[column], kde=True, bins='auto', color='steelblue')  # Automatically choose bin size
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

    # Plot boxplot to show spread and potential outliers
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[column], color='salmon')
    plt.title(f'Boxplot of {column}')
    plt.xlabel(column)

    # Adjust layout to prevent overlap and display the plots
    plt.tight_layout()
    plt.show()

In [ ]:
# hydrogen.csv hydrogen_small.csv kick.csv Prepare_SQuAD_tiny.ipynb tfidf_features.csv tfidf_features_small.csv tsdm.csv
kickdf = pd.read_csv(r'C:\Users\stevi\OneDrive\Desktop\Assignment 2\Datasets\kick.csv')

In [ ]:
kickdf.info()

In [ ]:
# get more information from VehOdo
print(kickdf['VehOdo'].describe())
print(kickdf['VehOdo'].value_counts())

In [ ]:
# get more information from VehOdo
print(kickdf['MMRAcquisitionAuctionAveragePrice'].describe())
print(kickdf['MMRAcquisitionAuctionAveragePrice'].value_counts())

# why is there a 0

In [ ]:
inspect_quantitative_variable(kickdf, 'MMRAcquisitionAuctionAveragePrice')
#Zero values      : 502

In [ ]:
# replace '0' with NaN as 0 is technically a blank entry
kickdf['MMRAcquisitionAuctionAveragePrice'] = kickdf['MMRAcquisitionAuctionAveragePrice'].replace([0], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
kickdf['MMRAcquisitionAuctionAveragePrice'] = (
    kickdf['MMRAcquisitionAuctionAveragePrice']
    .fillna(round(kickdf['MMRAcquisitionAuctionAveragePrice'].median()))
    .astype('int64')
)

In [ ]:
inspect_quantitative_variable(kickdf, 'MMRAcquisitionAuctionAveragePrice')

In [ ]:
# get more information from VehOdo
print(kickdf['Make'].describe())
print(kickdf['Make'].value_counts())

In [ ]:
print(kickdf['Make'].unique())

In [ ]:
# get more information from VehOdo
print(kickdf['WarrantyCost'].describe())
print(kickdf['WarrantyCost'].value_counts())

In [ ]:
# get more information from VehOdo
print(kickdf['IsBadBuy'].describe())
print(kickdf['IsBadBuy'].value_counts())

In [ ]:
kickdf.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
# take 3 variables and drop the rest. copy the dataframe to avoid warnings later
kickdf2 = kickdf[['VehOdo', 'MMRAcquisitionAuctionAveragePrice', 'WarrantyCost']].copy()
# convert df2 to matrix
X = kickdf2.to_numpy()
# scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans
# list to save the clusters and cost
clusters = []
inertia_vals = []
for k in range(2, 12, 1):
 # train clustering with the specified K
 model = KMeans(n_clusters=k, random_state=random_state)
 model.fit(X)

 # append model to cluster list
 clusters.append(model)
 inertia_vals.append(model.inertia_)

# plot the inertia vs K values
plt.plot(range(2,12,1), inertia_vals, marker='*')
plt.title("Elbow Method for Optimal K", fontsize=12)
plt.xlabel("Number of Clusters (K)", fontsize=11)
plt.ylabel("Inertia", fontsize=11)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
print(clusters[0])
print("Silhouette score for k=2", silhouette_score(X, clusters[0].predict(X)))
print(clusters[1])
print("Silhouette score for k=3", silhouette_score(X, clusters[1].predict(X)))
print(clusters[2])
print("Silhouette score for k=4", silhouette_score(X, clusters[2].predict(X)))

In [ ]:
from sklearn.cluster import KMeans
random_state = 10
# set the random state. different random state seeds might result in different centroids locations
model = KMeans(n_clusters=3, random_state=random_state)
model.fit(X)
# sum of intra-cluster distances
print("Sum of intra-cluster distance:", model.inertia_)
print("Centroid locations:")
for centroid in model.cluster_centers_:
 print(centroid)

In [ ]:
# set a different n_clusters
model = KMeans(n_clusters=3, random_state=random_state)
model.fit(X)
# sum of intra-cluster distances
print("Sum of intra-cluster distance:", model.inertia_)
print("Centroid locations:")
for centroid in model.cluster_centers_:
 print(centroid)
# add column for cluster ID to data
y = model.predict(X)
kickdf2['Cluster_ID'] = y
# how many in each?
print(kickdf2['Cluster_ID'].value_counts())
# pairplot
cluster_g = sns.pairplot(kickdf2, hue='Cluster_ID', diag_kind='hist')
plt.show()


In [ ]:
def inspect_clusters(df, cols, clusters_to_inspect):
 # prepare the column and bin size. Increase bin size to be more specific, but 20 is more than enough
    n_bins = 20

    for cluster in clusters_to_inspect:
        print(f"Distribution for cluster {cluster}")

        # create subplots
        fig, ax = plt.subplots(nrows=len(cols), figsize=(6, 10))
        fig.suptitle(f"Cluster {cluster}", fontsize=14)

        for j, col in enumerate(cols):
            # create the bins
            bins = np.linspace(df[col].min(), df[col].max(), n_bins)

            # plot distribution of the cluster using histogram
            sns.histplot(df[df['Cluster_ID'] == cluster][col],
                         bins=bins, ax=ax[j], kde=True, stat="density")

            # plot the normal distribution with a black line
            sns.kdeplot(df[col], ax=ax[j], color='k')

            ax[j].set_xlabel(col)
            ax[j].legend()

        plt.tight_layout()
        plt.show()

In [ ]:
inspect_clusters(kickdf2, ['VehOdo', 'MMRAcquisitionAuctionAveragePrice', 'WarrantyCost'], [0, 1, 2])


In [ ]:
# One-hot encode the 'make' column
make_dummies = pd.get_dummies(kickdf['Make'], prefix='Make')
kickdf3 = pd.concat([kickdf2, make_dummies], axis=1)

kickdf3.info()

In [ ]:
# Select only numeric columns for clustering
X3 = kickdf3.drop(columns=['Cluster_ID'])

# Standardise
scaler = StandardScaler()
X3_scaled = scaler.fit_transform(X3)
X3_scaled = pd.DataFrame(X3_scaled, columns=X3.columns)


X3_scaled.info()

In [ ]:
from sklearn.cluster import KMeans
random_state = 10
# set the random state. different random state seeds might result in different centroids locations
model = KMeans(n_clusters=3, random_state=random_state)
model.fit(X3_scaled)
# sum of intra-cluster distances
print("Sum of intra-cluster distance:", model.inertia_)
print("Centroid locations:")
for centroid in model.cluster_centers_:
 print(centroid)

In [ ]:
# fit KMeans
model = KMeans(n_clusters=3, random_state=random_state).fit(X3_scaled)

# attach cluster IDs to kickdf3
kickdf3['Cluster_ID_With_Make'] = model.labels_

# pairplot only for the 3 continuous features
sns.pairplot(
    kickdf3,
    vars=['VehOdo', 'MMRAcquisitionAuctionAveragePrice', 'WarrantyCost'],
    hue='Cluster_ID_With_Make',
    diag_kind='hist',
)
plt.show()


#Task 3

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
# hydrogen.csv hydrogen_small.csv kick.csv Prepare_SQuAD_tiny.ipynb tfidf_features.csv tfidf_features_small.csv tsdm.csv
tsdmdf = pd.read_csv(r'C:\Users\stevi\OneDrive\Desktop\Assignment 2\Datasets\tsdm.csv')

In [ ]:
tsdmdf.info()
tsdmdf.isnull().sum()
# observation date is appearing as object rather than datetime

In [ ]:
tsdmdf

In [ ]:
# convert OBSERVATION_DATE to datetime format
tsdmdf['OBSERVATION_DATE'] = pd.to_datetime(tsdmdf['OBSERVATION_DATE'])

# sort by date
tsdmdf = tsdmdf.sort_values(['PADDOCK_ID', 'OBSERVATION_DATE']).reset_index(drop=True)

In [ ]:
inspect_qualitative_variable(tsdmdf, 'OBSERVATION_DATE')

In [ ]:
inspect_quantitative_variable(tsdmdf, 'TSDM')

In [ ]:
inspect_quantitative_variable(tsdmdf, '15D_AVG_DAILY_RAIN')

In [ ]:
inspect_quantitative_variable(tsdmdf, '15D_AVG_MIN_TEMP')

In [ ]:
inspect_quantitative_variable(tsdmdf, '15D_AVG_RH_TMAX')

In [ ]:
inspect_quantitative_variable(tsdmdf, '15D_AVG_RH_TMIN')

In [ ]:
inspect_quantitative_variable(tsdmdf, '15D_AVG_EVAP_SYN')

In [ ]:
inspect_quantitative_variable(tsdmdf, '15D_AVG_RADIATION')

In [ ]:
tsdmdf

In [ ]:
seq_counts = tsdmdf.groupby('PADDOCK_ID').size()
print(seq_counts.describe())

In [ ]:
def create_sequences(sequence, lookback, forecast_horizon, target_col):
    T, num_features = sequence.shape
    X, y, lengths = [], [], []
    pad_vector = np.zeros((lookback, num_features))

    # Fixed-length lookback with pre-padding
    for t in range(1, T - forecast_horizon + 1):
        context = sequence[:t]
        if len(context) > lookback:
            context = context[-lookback:]

        padded_context = pad_vector.copy()
        padded_context[-len(context):] = context

        X.append(padded_context)
        y.append(sequence[t:t + forecast_horizon, target_col])
        lengths.append(min(len(context), lookback))

    return np.array(X), np.array(y), lengths


In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

def data_prep(df, feature_columns, lookback, test_steps, target_col, forecast_horizon):
    X_all, y_all = [], []
    train_data = []
    test_data = []
    lengths_all = []

    all_train_values = []
    for _, group in df.groupby("PADDOCK_ID"):
        feature_values = group[feature_columns].values
        if len(feature_values) > lookback + test_steps:
            all_train_values.append(feature_values[:-test_steps])

    all_train_values = np.vstack(all_train_values)
    global_scaler = MinMaxScaler()
    global_scaler.fit(all_train_values)

    for paddock_id, group in df.groupby('PADDOCK_ID'):
        feature_values = group[feature_columns].values

        if len(feature_values) <= 194:
            continue

        train_sample = global_scaler.transform(feature_values[:-test_steps])
        test_sample  = global_scaler.transform(feature_values[-test_steps:])

        train_data.append((paddock_id, train_sample))
        test_data.append((paddock_id, test_sample, global_scaler))

        X_location, y_location, lengths = create_sequences(
            train_sample, lookback, forecast_horizon, target_col
        )

        X_all.append(X_location)
        y_all.append(y_location)
        lengths_all.append(lengths)

    X_all = np.concatenate(X_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)
    lengths_all = np.concatenate(lengths_all, axis=0)

    X_all = X_all.reshape((X_all.shape[0], X_all.shape[1], X_all.shape[2]))

    return (
        torch.Tensor(X_all),
        torch.Tensor(y_all),
        torch.Tensor(lengths_all),
        train_data,
        test_data,
    )


In [ ]:
lookback   = 5
forecast_horizon = 5
test_steps = 5
feature_columns = ['TSDM']  # univariate
target_col = feature_columns.index('TSDM')

X_5, y_5, lengths_5, train_d_5, test_d_5 = data_prep(
    tsdmdf, feature_columns, lookback, test_steps, target_col, forecast_horizon
)

print("Shape of input data after sequence creation:", X_5.shape)
print("Shape of targets after sequence creation:", y_5.shape)

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MyLSTMNet(nn.Module):
    def __init__(self, num_features, hidden_layer_size, num_layers, output_size, dropout_prob):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_layer_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_layer_size, output_size)

    def forward(self, data, lengths):
        """
        Forward pass for variable-length LSTM input.
        - data: tensor [batch, time, features]
        - lengths: tensor of sequence lengths (for packing)
        """
        # Pack the input sequence so padded timesteps are ignored
        packed_data = pack_padded_sequence(
            data, lengths.cpu(), batch_first=True, enforce_sorted=False
        )

        # Pass through LSTM
        packed_output, (hn, cn) = self.lstm(packed_data)

        # Take the hidden state from the last layer (num_layers - 1)
        last_hidden = hn[-1]  # shape: [batch, hidden_layer_size]

        # Dropout + final fully connected layer to produce output
        out = self.dropout(last_hidden)
        out = self.fc(out)    # shape: [batch, output_size]
        return out


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, random_split

def train_predict_model(model, n_epochs, lr, X_all, y_all, lengths, validation_split, batch_size):
    # Split data into train and validation sets
    dataset = TensorDataset(X_all, y_all, lengths)
    val_size = int(len(dataset) * validation_split)
    train_size = len(dataset) - val_size
    train_set, val_set = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print(f"The model has {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters")

    train_loss_history = []
    val_loss_history = []

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(n_epochs):
        model.train()
        for X_batch, y_batch, lengths_batch in train_loader:
            y_pred = model(X_batch, lengths_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation check every 100 epochs
        if epoch % 100 == 0:
            model.eval()
            with torch.no_grad():
                train_preds = model(X_all[train_set.indices], lengths[train_set.indices])
                train_loss = loss_fn(train_preds, y_all[train_set.indices]).item()

                val_preds = model(X_all[val_set.indices], lengths[val_set.indices])
                val_loss = loss_fn(val_preds, y_all[val_set.indices]).item()

                print(f"Epoch {epoch+1}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")

                train_loss_history.append(train_loss)
                val_loss_history.append(val_loss)

                # Save best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model_state = model.state_dict()

    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    return train_loss_history, val_loss_history, model


In [ ]:
import itertools, torch

# Parameter grid
hidden_sizes = [10, 20]
num_layers = [1, 2]
dropouts = [0.1, 0.2]
lrs = [0.001]
batch_sizes = [32, 64]

n_epochs = 200
validation_split = 0.2

best_val_loss = float('inf')
best_cfg = None

for hs, nl, do, lr, bs in itertools.product(hidden_sizes, num_layers, dropouts, lrs, batch_sizes):
    model = MyLSTMNet(
        num_features=X_5.shape[2],
        hidden_layer_size=hs,
        num_layers=nl,
        output_size=5,
        dropout_prob=do
    )

    train_hist, val_hist, _ = train_predict_model(
        model, n_epochs, lr, X_5, y_5, lengths_5, validation_split, bs
    )

    val_loss = val_hist[-1] if val_hist else float('inf')
    print(f"hs={hs}, nl={nl}, do={do}, lr={lr}, bs={bs} -> val_loss={val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_cfg = dict(hidden_size=hs, num_layers=nl, dropout=do, lr=lr, batch_size=bs)

print("Best parameters:", best_cfg, "with val_loss:", best_val_loss)


In [ ]:
# Optimal hyperparameters (from tuning)
hidden_layer_size = 10
num_layers = 2
dropout_prob = 0.2
lr = 0.001
batch_size = 32
n_epochs = 501
validation_split = 0.2

# Rebuild and train
num_features = X_5.shape[2]
output_size = 5

model_lstm_5 = MyLSTMNet(
    num_features=num_features,
    hidden_layer_size=hidden_layer_size,
    num_layers=num_layers,
    output_size=output_size,
    dropout_prob=dropout_prob
)
train_loss_history_5, val_loss_history_5, model_lstm_5 = train_predict_model(
    model_lstm_5,
    n_epochs,
    lr,
    X_5,
    y_5,
    lengths_5,
    validation_split,
    batch_size
)

In [ ]:
def vis_train_loss(train_loss_history, val_loss_history):
 epochs = range(0, n_epochs, 100)
 plt.plot(epochs, train_loss_history, label='Training Loss')
 plt.plot(epochs, val_loss_history, label='Validation Loss') # <-- add this
 plt.xlabel('Epochs')
 plt.ylabel('Loss')
 plt.title('Loss Convergence')
 plt.legend()
 plt.grid()
 plt.show()

In [ ]:
vis_train_loss(train_loss_history_5, val_loss_history_5)


In [ ]:
from sklearn.metrics import root_mean_squared_error, r2_score
import torch
import numpy as np
import matplotlib.pyplot as plt

def pred_eval(model, X, y, lengths, train_d, test_d, lookback, target_col):
    model.eval()
    with torch.no_grad():
        # Training metrics
        train_preds = model(X, lengths)
        print("Training RMSE:", root_mean_squared_error(y.flatten().tolist(),
                                                       train_preds.flatten().tolist()))
        print("Training R2:", r2_score(y.flatten().tolist(),
                                       train_preds.flatten().tolist()))


        X_test = []
        y_test = []
        lengths_test = []
        for count, (location_id, test_values, scaler) in enumerate(test_d):
            train_values = train_d[count][1]
            X_test.append(train_values[-lookback:])
            y_test.append(test_values[:, target_col])
            lengths_test.append(len(train_values[-lookback:]))

        X_test = torch.Tensor(np.array(X_test))
        y_test = torch.Tensor(np.array(y_test))
        lengths_test = torch.Tensor(lengths_test).long()

        # Predictions & test metrics
        test_preds = model(X_test, lengths_test)
        print("Test RMSE:", root_mean_squared_error(y_test.flatten().tolist(),
                                                    test_preds.flatten().tolist()))
        print("Test R2:", r2_score(y_test.flatten().tolist(),
                                   test_preds.flatten().tolist()))


        plt.figure(figsize=(10, 6))
        plt.plot(y_test.flatten().tolist(), label="Expected Value")
        plt.plot(test_preds.flatten().tolist(), label="Predicted Value")
        plt.grid()
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()


In [ ]:
lookback = 5
target_col = 0
pred_eval(model_lstm_5, X_5, y_5, lengths_5, train_d_5, test_d_5, lookback, target_col)

In [ ]:
lookback   = 10
forecast_horizon = 5
test_steps = 5
feature_columns = ['TSDM']  # univariate
target_col = feature_columns.index('TSDM')

X_10, y_10, lengths_10, train_d_10, test_d_10 = data_prep(
    tsdmdf, feature_columns, lookback, test_steps, target_col, forecast_horizon
)

print("Shape of input data after sequence creation:", X_10.shape)
print("Shape of targets after sequence creation:", y_10.shape)

In [ ]:
# Optimal hyperparameters (from tuning)
hidden_layer_size = 10
num_layers = 2
dropout_prob = 0.2
lr = 0.001
batch_size = 32
n_epochs = 501
validation_split = 0.2

# Rebuild and train
num_features = X_10.shape[2]
output_size = 5

model_lstm_10 = MyLSTMNet(
    num_features=num_features,
    hidden_layer_size=hidden_layer_size,
    num_layers=num_layers,
    output_size=output_size,
    dropout_prob=dropout_prob
)
train_loss_history_10, val_loss_history_10, model_lstm_10 = train_predict_model(
    model_lstm_10,
    n_epochs,
    lr,
    X_10,
    y_10,
    lengths_10,
    validation_split,
    batch_size
)

In [ ]:
print(model_lstm_10)

In [ ]:
n_epochs = 501
lr = 0.001
validation_split = 0.2
batch_size = 32

train_loss_history_10, val_loss_history_10, model_lstm_10 = train_predict_model(
    model_lstm_10,
    n_epochs,
    lr,
    X_10,
    y_10,
    lengths_10,
    validation_split,
    batch_size
)

In [ ]:
vis_train_loss(train_loss_history_10, val_loss_history_10)

In [ ]:
lookback = 10
target_col = 0
pred_eval(model_lstm_10, X_10, y_10, lengths_10, train_d_10, test_d_10, lookback, target_col)

In [ ]:
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

def create_sequences_nonfixedlookback(sequence, lookback, forecast_horizon, target_col, pad_value=0.0):
    T, num_features = sequence.shape
    X, y, lengths = [], [], []

    if lookback > 0:
        pad_vector = np.zeros((lookback, num_features), dtype=np.float32)

        for t in range(1, T - forecast_horizon + 1):

            context = sequence[:t]
            if len(context) > lookback:
                context = context[-lookback:]


            padded_context = pad_vector.copy()
            padded_context[-len(context):] = context

            X.append(padded_context)
            y.append(sequence[t:t + forecast_horizon, target_col])
            lengths.append(min(len(context), lookback))

        return (np.array(X, dtype=np.float32),
                np.array(y, dtype=np.float32),
                np.array(lengths, dtype=np.int64))

    else:
        for t in range(1, T - forecast_horizon + 1):
            context = torch.tensor(sequence[:t], dtype=torch.float32)
            X.append(context)
            y.append(torch.tensor(sequence[t:t + forecast_horizon, target_col], dtype=torch.float32))
            lengths.append(len(context))

        X_padded = pad_sequence(X, batch_first=True, padding_value=float(pad_value))
        y_tensor = torch.stack(y)

        return (X_padded.numpy(),
                y_tensor.numpy(),
                np.array(lengths, dtype=np.int64))


In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

def data_prep_nonfixedlookback(df, feature_columns, lookback, test_steps, target_col, forecast_horizon):
    X_all, y_all = [], []
    train_data = []
    test_data = []
    lengths_all = []

    all_train_values = []
    for _, group in df.groupby("PADDOCK_ID"):
        feature_values = group[feature_columns].values
        if len(feature_values) > lookback + test_steps:

            all_train_values.append(feature_values[:-test_steps])

    all_train_values = np.vstack(all_train_values)
    global_scaler = MinMaxScaler()
    global_scaler.fit(all_train_values)

    for paddock_id, group in df.groupby('PADDOCK_ID'):
        feature_values = group[feature_columns].values

        if len(feature_values) <= 194:
            continue

        train_sample = global_scaler.transform(feature_values[:-test_steps])
        test_sample  = global_scaler.transform(feature_values[-test_steps:])

        train_data.append((paddock_id, train_sample))
        test_data.append((paddock_id, test_sample, global_scaler))

        X_location, y_location, lengths = create_sequences_nonfixedlookback(
            train_sample, lookback, forecast_horizon, target_col
        )

        X_all.append(X_location)
        y_all.append(y_location)
        lengths_all.append(lengths)

    X_all = np.concatenate(X_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)
    lengths_all = np.concatenate(lengths_all, axis=0)


    X_all = X_all.reshape((X_all.shape[0], X_all.shape[1], X_all.shape[2]))

    return (
        torch.Tensor(X_all),
        torch.Tensor(y_all),
        torch.Tensor(lengths_all),
        train_data,
        test_data,
    )


In [ ]:
lookback   = 0 # dont restrict
forecast_horizon = 5
test_steps = 5
# multivairate (predict TSDM)
feature_columns = ['TSDM', '15D_AVG_DAILY_RAIN','15D_AVG_MAX_TEMP',
                   '15D_AVG_MIN_TEMP','15D_AVG_RH_TMAX','15D_AVG_RH_TMIN',
                   '15D_AVG_EVAP_SYN','15D_AVG_RADIATION']

target_col = feature_columns.index('TSDM')

X_m, y_m,lengths_m, train_d_m, test_d_m = data_prep_nonfixedlookback(
    tsdmdf, feature_columns, lookback, test_steps, target_col, forecast_horizon
)


print("Shape of input data after sequence creation:", X_m.shape)
print("Shape of targets after sequence creation:", y_m.shape)

In [ ]:
# Optimal hyperparameters (from tuning)
hidden_layer_size = 10
num_layers = 2
dropout_prob = 0.2
lr = 0.001
batch_size = 32
n_epochs = 501
validation_split = 0.2

# Rebuild and train
num_features = X_m.shape[2]
output_size = test_steps

model_lstm_m = MyLSTMNet(
    num_features=num_features,
    hidden_layer_size=hidden_layer_size,
    num_layers=num_layers,
    output_size=output_size,
    dropout_prob=dropout_prob
)
train_loss_history_m, val_loss_history_m, model_lstm_m = train_predict_model(
    model_lstm_m,
    n_epochs,
    lr,
    X_m,
    y_m,
    lengths_m,
    validation_split,
    batch_size
)

In [ ]:
vis_train_loss(train_loss_history_m, val_loss_history_m)

In [ ]:
target_col = 0
pred_eval(model_lstm_m, X_m, y_m, lengths_m, train_d_m, test_d_m, lookback, target_col)

#Task 4

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, roc_curve
import torch
from transformers import BertweetTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, EarlyStoppingCallback
from datasets import Dataset
import matplotlib.pyplot as plt
import seaborn as sns

# hydrogen.csv hydrogen_small.csv kick.csv Prepare_SQuAD_tiny.ipynb tfidf_features.csv tfidf_features_small.csv tsdm.csv
df = pd.read_csv(r'C:\Users\stevi\OneDrive\Desktop\Assignment 2\Datasets\hydrogen_small.csv')

In [ ]:
df.info()

In [ ]:
tfidf.info()

In [ ]:
df["label"].unique()


In [ ]:
df["text"].iloc[18]


In [ ]:
def clean_message(text):
 # Remove URLs from the message. Match anything starting wtih "http", "https", or "www"
  # followed by any non-whitespace characters (\S).
 text = re.sub(r"http\S+|www\S+|https\S+", "", text)
 # Remove any tags from the message. Matches the '@' symbol followed by any letters,
 # numbers and underscores (\w).
 text = re.sub(r"@\w+", "", text)
 # Remove any non-ASCII characters from the text, such as emojis.
 # If you were dealing with a dataset containing non-English/Latin text, then you would
 # want to change this regular expression.
 text = re.sub(r"[^\x00-\x7F]+", "", text)
 # Replace any sequences of consecutive whitespace with a single space character.
 # This includes newlines and spaces.

 text = re.sub(r"\$(\w+)", r"\1", text)
 # Normalize cashtags by removing the '$' but keeping the company name

 text = re.sub(r"\s+", " ", text)
 # Finally, remove any leading or trailing whitespace with the strip() function.
 return text.strip()



In [ ]:
df["text"] = df["text"].apply(clean_message)

In [ ]:
df["text"].iloc[18]

In [ ]:
df["label"] = df["label"].map({
 'Irrelevant': 0, # Negative = 0
 'Relevant': 1 # Positive = 1
})


In [ ]:
df["label"].unique()

In [ ]:
df[df["label"] == 0].head(5)

In [ ]:
df[df["label"] == 1].head(5)

In [ ]:
df["label"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X = df["text"].values
y = df["label"].values

random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=random_state
)
print("Training size:", len(X_train))
print("Testing size:", len(X_test))

In [ ]:
df_hydro = pd.DataFrame({"text": X, "label": y})
df_hydro["label"].value_counts()

In [ ]:
train_df = pd.DataFrame({"text": X_train, "label": y_train})
test_df = pd.DataFrame({"text": X_test, "label": y_test})
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [ ]:
print("Train dataset:", train_ds)
print("Test dataset:", test_ds)

In [ ]:
model_name = "vinai/bertweet-base"
tokenizer = BertweetTokenizer.from_pretrained(model_name)

In [ ]:
# Function that is applied to all samples in the dataset.
def tokenize(batch):
 # We set truncation=True to truncate (cut off) messages that are too long.
 # NOTE: Not all models require this, you may get a warning indicating that it has no effect.
 # Padding is set to True if the model requires a fixed sequence length.
 return tokenizer(batch['text'], truncation=True, padding=True)
# Apply to both the training and testing datasets.
# We set batched to True which can enable parallel processing, however on my machine I found
# it did not scale to a greater number of threads.
train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

In [ ]:
train_ds

In [ ]:
# Ensure the resources for any existing model has been freed.
try:
 del model
except NameError:
 pass
# Download/load the base model. We use the "vinai/bertweet-base" model here.
# Set the number of labels to the number of unique labels in the dataframe, which is 2.
# Set the problem type to single label classification, since we want one class for each sample.
modelBERTweet = RobertaForSequenceClassification.from_pretrained(
 model_name,
 num_labels=df_hydro["label"].nunique(),
 problem_type="single_label_classification")


In [ ]:
def compute_metrics(pred):
 labels = pred.label_ids
 preds = pred.predictions.argmax(-1)
 acc = accuracy_score(labels, preds)
 prec, recall, f1, _ = precision_recall_fscore_support(
 labels, preds, average="binary", pos_label=1)
 return {
 "accuracy": acc,
 "precision": prec,
 "recall": recall,
 "f1": f1
 }

In [ ]:
training_args = TrainingArguments(
 output_dir="./results",
 num_train_epochs=50,
 per_device_train_batch_size=16,
 per_device_eval_batch_size=64,
 eval_strategy="epoch",
 save_strategy="epoch",
 learning_rate=1e-5,
 weight_decay=0.01,
 logging_dir="./logs",
 logging_steps=10,
 # Added for early stopping.
 metric_for_best_model = "loss",
 load_best_model_at_end = True,
 seed = 42
)

In [ ]:
# Switch the model to training mode, enabling dropout etc layers.
modelBERTweet.train()
trainerBERTweet = Trainer(
 model=modelBERTweet,
 args=training_args,
 train_dataset=train_ds,
 eval_dataset=test_ds,
 processing_class=tokenizer,
 data_collator=DataCollatorWithPadding(tokenizer),
 compute_metrics=compute_metrics,
 callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)
trainerBERTweet.train()

In [ ]:
# Switch the model to evaluation mode, disabling dropout etc layers.
modelBERTweet.eval()
# Evaluate the datasets.
train_results = trainerBERTweet.evaluate(train_ds)
test_results = trainerBERTweet.evaluate(test_ds)

In [ ]:
def display_evaluation(setname, results):
 print(f"{setname} Set Accuracy:", round(results["eval_accuracy"], 3))
 print(f"{setname} Set Precision:", round(results["eval_precision"], 3))
 print(f"{setname} Set Recall:", round(results["eval_recall"], 3))
 print(f"{setname} Set F1 score:", round(results["eval_f1"], 3))
display_evaluation("Training", train_results)
display_evaluation("Testing", test_results)

In [ ]:
# Returns (matrix, tokens)
def compute_attention_matrix(tokenizer, model, text):
  # Feed into the model, you could also grab the token embedding directly
  # from the dataset, in which case this step would be unnecessary. We want
  # the output in Tensor format that we can feed to the model, so we use
  # return_tensors="pt" (PyTorch Tensor). Lastly, send the tensor to
  # whichever device the model is located on. This is unnecessary if you
  # are running purely on the CPU, but needed for models on GPUs.
  tokens = tokenizer(text, return_tensors="pt").to(model.device)
  # We use torch.no_grad() to ensure the weights in the model are unchanged.
  with torch.no_grad():
    pred = model(**tokens, output_attentions=True)
  # Stack layers. Depending on your model, this may have no effect.
  # Move it back to the GPU if it was previously on the GPU.
  attentions = torch.stack(pred.attentions).cpu()
  # Remove the batch dimension, as there is only a zero value there.
  attentions = attentions.squeeze(1)
  # Average over the transformer layers and heads.
  attentions = attentions.mean(dim=0).mean(dim=0)
  # attentions now contains a matrix of importance from every token to every
  # other token. e.g. if the message contained 10 tokens, it would be 10x10.
  # Select the predicted class.
  pred_class = pred.logits.cpu().argmax(-1).item()
  # Also return a string representation of the tokens in the message.
  # Plotting the integer token IDs would not be very meaningful.
  token_strs = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
  return (attentions, pred_class, token_strs)

In [ ]:
#pred = trainer.predict(test_ds)

In [ ]:
# Convert to PyTorch tensor, apply softmax, and convert back to a numpy array.
# pred_probs_bertweet = torch.nn.functional.softmax(torch.Tensor(pred.predictions)).numpy()

In [ ]:
pred_bertweet = trainerBERTweet.predict(test_ds)
y_test = pred_bertweet.label_ids

probs_bertweet = torch.nn.functional.softmax(
    torch.tensor(pred_bertweet.predictions), dim=1
).numpy()

auc_bertweet = roc_auc_score(y_test, probs_bertweet[:, 1])
fpr_bertweet, tpr_bertweet, _ = roc_curve(y_test, probs_bertweet[:, 1])


In [ ]:
from sklearn.metrics import classification_report

y_true = pred_bertweet.label_ids  # true labels
y_pred = pred_bertweet.predictions.argmax(-1)  # predicted labels

print(classification_report(y_true, y_pred, digits=3))

In [ ]:
model_name = "Twitter/twhin-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Ensure the resources for any existing model has been freed.
try:
 del model
except NameError:
 pass
# Download/load the base model. We use the "vinai/bertweet-base" model here.
# Set the number of labels to the number of unique labels in the dataframe, which is 2.
# Set the problem type to single label classification, since we want one class for each sample.
modelTwHIN = AutoModelForSequenceClassification.from_pretrained(
 model_name,
 num_labels=df_hydro["label"].nunique(),
 problem_type="single_label_classification")

In [ ]:
training_args = TrainingArguments(
 output_dir="./results",
 num_train_epochs=50,
 per_device_train_batch_size=16,
 per_device_eval_batch_size=64,
 eval_strategy="epoch",
 save_strategy="epoch",
 learning_rate=1e-5,
 weight_decay=0.01,
 logging_dir="./logs",
 logging_steps=10,
 # Added for early stopping.
 metric_for_best_model = "loss",
 load_best_model_at_end = True,
 seed=42
)

In [ ]:
# Switch the model to training mode, enabling dropout etc layers.
modelTwHIN.train()
trainerTwHIN = Trainer(
 model=modelTwHIN,
 args=training_args,
 train_dataset=train_ds,
 eval_dataset=test_ds,
 processing_class=tokenizer,
 data_collator=DataCollatorWithPadding(tokenizer),
 compute_metrics=compute_metrics,
 callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)
trainerTwHIN.train()

In [ ]:
# Switch the model to evaluation mode, disabling dropout etc layers.
modelTwHIN.eval()
# Evaluate the datasets.
train_results = trainerTwHIN.evaluate(train_ds)
test_results = trainerTwHIN.evaluate(test_ds)

In [ ]:
def display_evaluation(setname, results):
 print(f"{setname} Set Accuracy:", round(results["eval_accuracy"], 3))
 print(f"{setname} Set Precision:", round(results["eval_precision"], 3))
 print(f"{setname} Set Recall:", round(results["eval_recall"], 3))
 print(f"{setname} Set F1 score:", round(results["eval_f1"], 3))
display_evaluation("Training", train_results)
display_evaluation("Testing", test_results)

In [ ]:
# Convert to PyTorch tensor, apply softmax, and convert back to a numpy array.
# pred_probs_twhin = torch.nn.functional.softmax(torch.Tensor(pred.predictions)).numpy()

In [ ]:
pred_twhin = trainerTwHIN.predict(test_ds)
probs_twhin = torch.nn.functional.softmax(
    torch.tensor(pred_twhin.predictions), dim=1
).numpy()

auc_twhin = roc_auc_score(y_test, probs_twhin[:, 1])
fpr_twhin, tpr_twhin, _ = roc_curve(y_test, probs_twhin[:, 1])

In [ ]:
import numpy as np
print("Means:", probs_bertweet[:,1].mean(), probs_twhin[:,1].mean())
print("Arrays equal?", np.allclose(probs_bertweet, probs_twhin))  # should be False

In [ ]:
plt.figure(figsize=(6,5))
plt.plot(fpr_bertweet, tpr_bertweet, label=f"BERTweet (AUC = {auc_bertweet:.3f})", lw=1.0)
plt.plot(fpr_twhin,    tpr_twhin,    label=f"TwHIN-BERT (AUC = {auc_twhin:.3f})", lw=1.0)
plt.plot([0,1],[0,1], linestyle="--", color="navy", lw=0.5)
plt.xlim([0,1]); plt.ylim([0,1])
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Hydrogen Tweet Classification")
plt.legend(loc="lower right"); plt.grid(alpha=0.3)
plt.show()

In [ ]:
def plot_attention(attentions, tokens, title):
 # Enlarge figure to take up more of the width.
 plt.figure(figsize=(10, 8))
 plt.title(title)
 # Plot heatmap.
 sns.heatmap(
 attentions, # Plot our attention matrix.
 xticklabels=tokens, # Display token names on X axis.
 yticklabels=tokens, # Display token names on Y axis.
 cmap='binary', # Black for low, white for high
 cbar=True # Display colour bar.
 )

 plt.show()

In [ ]:
def display_attention_matrix(tokenizer, model, text, pred_label):
    attention, pred_class, tokens = compute_attention_matrix(tokenizer, model, text)
    pred_label = pred_label
    plot_attention(attention, tokens, text + f"\nPredicted class: {pred_label}")

In [ ]:
tokenizerBT   = BertweetTokenizer.from_pretrained("vinai/bertweet-base")
tokenizerTWHIN = AutoTokenizer.from_pretrained("Twitter/twhin-bert-base")

In [ ]:
display_attention_matrix(tokenizerBT, modelBERTweet, "BERTweet Model: " + df[df["label"] == 1].iloc[0]["text"], "Positive")

In [ ]:
display_attention_matrix(tokenizerTWHIN, modelTwHIN, "TwHIN-BERT Model: " + df[df["label"] == 1].iloc[0]["text"], "Positive")

In [ ]:
display_attention_matrix(tokenizerBT, modelBERTweet, "BERTweet Model: " + df[df["label"] == 0].iloc[0]["text"], "Negative")

In [ ]:
display_attention_matrix(tokenizerTWHIN, modelTwHIN, "TwHIN-BERT Model: " + df[df["label"] == 0].iloc[0]["text"], "Negative")

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, accuracy_score, precision_recall_fscore_support

tfidf = pd.read_csv(r'C:\Users\stevi\OneDrive\Desktop\Assignment 2\Datasets\tfidf_features_small.csv')
y = df_hydro['label'].astype(int).values
X = tfidf.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("Size of training set:", len(X_train))
print("Size of testing set:", len(X_test))

In [ ]:
logreg = LogisticRegression(
    solver="lbfgs", #default
    penalty="l2", #default
    C=1.0, #default
    max_iter=100, #default
    random_state=42
)
logreg.fit(X_train, y_train)

print("\n=== Logistic Regression (TF-IDF) — baseline ===")
print("Train accuracy:", logreg.score(X_train, y_train))
print("Test  accuracy:", logreg.score(X_test, y_test))


In [ ]:
y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))

In [ ]:
# ROC-AUC
y_prob = logreg.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
fpr_logreg, tpr_logreg, _ = roc_curve(y_test, y_prob)
print("ROC-AUC (baseline):", round(auc, 3))

In [ ]:
from sklearn.metrics import roc_curve
plt.figure(figsize=(6, 5))

plt.plot(fpr_bertweet, tpr_bertweet,
         label=f"BERTweet (AUC = {auc_bertweet:.3f})",
         color='red', lw=1.0)

#plt.plot(fpr_twhin, tpr_twhin,
#        label=f"TwHIN-BERT (AUC = {auc_twhin:.3f})",
#        color='green', lw=1.0)

plt.plot(fpr_logreg, tpr_logreg,
         label=f"Logistic Regression (AUC = {auc:.3f})",
         color='blue', lw=1.0)

plt.plot([0, 1], [0, 1],
         linestyle='--', color='navy', lw=0.5, label='Chance')

plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Hydrogen Tweet Classification")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

#Task 5

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback, AutoTokenizer, AutoModelWithLMHead
from rouge_score import rouge_scorer
from tqdm import tqdm
from transformers import logging as transformers_logging

In [ ]:
# Set seed for reproducibility
torch.manual_seed(42)

In [ ]:
# 1. Load and preprocess SQuAD dataset
dataset = load_dataset("squad")

In [ ]:
print("Number of training examples:", len(dataset['train']))
dataset['train'][0]

In [ ]:
# Take subsets to avoid overload
train_dataset = dataset["train"].select(range(10000,11000))
val_dataset = dataset["validation"].select(range(3000,3100))
test_dataset = dataset["validation"].select(range(3100, 3200))  # No official SQuAD test set

In [ ]:
print("Size of training set:", len(train_dataset))
print("Size of validation set:", len(val_dataset))
print("Size of testing set:", len(test_dataset))

In [ ]:
MODEL_NAME = "t5-small"
#MODEL_NAME = "t5-base"
MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
def encode_question_and_context(question, context):
    return f"question: {question}  context: {context}"

# Obtains the context, question and answer from a given sample.
def extract_sample_parts(sample):
    context = sample["context"]
    question = sample["question"]
    answer = sample["answers"]['text'][0]
    question_with_context = encode_question_and_context(question, context)
    return (question_with_context, question, answer)

# Encodes the sample, returning token IDs.
def preprocess(sample):
    # Extract data from sample.
    question_with_context, question, answer = extract_sample_parts(sample)

    # Using truncation causes the tokenizer to emit a warning for every sample.
    # This will generate a significant amount of messages, and likely crash
    # your browser tab. We temporarily disable log messages to work around this.
    # See https://github.com/huggingface/transformers/issues/14285
    old_level = transformers_logging.get_verbosity()
    transformers_logging.set_verbosity_error()

    # Generate tokens for the input.
    # We include both the context and the question (first two parameters).
    input_tokens = tokenizer(question_with_context, question, padding="max_length",
                             truncation=True, max_length=MAX_INPUT_LENGTH)

    # Generate tokens for the expected answer. There is no need to include the
    output_tokens = tokenizer(answer, padding="max_length", truncation=True,
                              max_length=MAX_OUTPUT_LENGTH)

    # Restore old logging level, see above.
    transformers_logging.set_verbosity(old_level)

    # The output of the tokenizer is a map containing {input_ids, attention_mask}.
    # For trianing, we need to add the labels (answer/output tokens) to the map.
    input_tokens["labels"] = np.array(output_tokens["input_ids"])

    return input_tokens

In [ ]:
# Preprocess the datasets
training_set_enc = train_dataset.map(preprocess, batched=False)
validation_set_enc = val_dataset.map(preprocess, batched=False)
testing_set_enc = test_dataset.map(preprocess, batched=False)

In [ ]:
# Prepare 20 data points for qualitative analysis
test = testing_set_enc.select(range(20))
test

In [ ]:
# Prepare 20 data points for qualitative analysis
q_data = test_dataset.select(range(20))
q_data

In [ ]:
# Ensure the resources for any existing model has been freed.
try:
 del model
except NameError:
 pass
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
# We're interested in the input_ids (question/context tokens), attention mask,
# and labels (answer token IDs). Use PyTorch tensors for all three datasets.
columns = ["input_ids", "attention_mask", "labels"]
training_set_enc.set_format(type="torch", columns=columns)
validation_set_enc.set_format(type="torch", columns=columns)
testing_set_enc.set_format(type="torch", columns=columns)

In [ ]:
training_args = TrainingArguments(
 output_dir="./results",
 num_train_epochs=50,
 per_device_train_batch_size=8,
 per_device_eval_batch_size=8,
 eval_strategy="epoch",
 save_strategy="epoch",
 learning_rate=3e-4,
 weight_decay=0.01,
 save_total_limit=2,
 logging_dir="./logs",
 logging_steps=10,
 load_best_model_at_end=True,
 metric_for_best_model="eval_loss",
 greater_is_better=False
)

In [ ]:
# Switch the model to training mode, enabling dropout etc layers.
model.train()
trainer = Trainer(
 model=model,
 args=training_args,
 train_dataset=training_set_enc,
 eval_dataset=validation_set_enc,
 processing_class=tokenizer,
 data_collator=DataCollatorForSeq2Seq(tokenizer),
 callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
trainer.train()


In [ ]:
def display_evaluation(setname, results):
 print(f"{setname} Set Loss:", round(results["eval_loss"], 3))

# Switch the model to evaluation mode, disabling dropout etc layers.
model.eval()
# Evaluate the datasets.
display_evaluation("Training", trainer.evaluate(training_set_enc))
display_evaluation("Testing", trainer.evaluate(testing_set_enc))


In [ ]:
from itertools import batched
# Generates a response for a single input/question.
def generate_response(tokenizer, model, question):
 # Convert the sentences into a list of numeric tokens. We instruct the tokenizer
 # to return PyTorch tensors ("pt") so that we can feed them directly into the model.
 tokenized = tokenizer(question, return_tensors="pt", padding=True, truncation=True,
 max_length=MAX_OUTPUT_LENGTH).to(model.device)

 # Generate outputs using the model.
 with torch.no_grad():
  outputs = model.generate(**tokenized)
 # The model outputs a list of numeric tokens. To convert these tokens back to
 # sentences, we can use the batch_decode function from the tokenizer.
 outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
 return outputs

# Generates a list of responses from the specified model, optionally including
# the context in the prompt. If limit is set, then answers will only be generated
# for the first N questions of the dataset.
def generate_answers(tokenizer, model, dataset, use_context=True, limit=None):
 # Subsampling if requested.
 if limit is not None:
  dataset = dataset.select(range(limit))
 # Create list of encoded tokens, similarly to how we preprocessed the data for
 # training. We do this so we can use batch processing to speed up inference.
 questions = []
 inputs = []
 references = []
 for sample in dataset:
  question_with_context, question, answer = extract_sample_parts(sample)
  # Only include the context if the caller requested it.
  if use_context:
    inputs.append(question_with_context)
  else:
    inputs.append(question)
  # Include the original question/answer.
  questions.append(question)
  references.append(answer)
 # Generate responses for each of the prompts/inputs.
 # Submitting each question to the model separately would significantly
 # increase processing time, especially if the model is located on the GPU.
 # Instead, we group questions together in the same batch size that we used
 # for training.
 outputs = []
 for samples in batched(inputs, 128):
  # Python's batched() function returns a tuple of the batch
  # size, which we have to first convert to a list.
  responses = generate_response(tokenizer, model, list(samples))
  # generate_responses() returns an equal-sized list of responses.
  outputs.extend(responses)
 # The length of the reference responses should equal the length of the
 # generated responses.
 assert (len(outputs) == len(references))
 return outputs, references, questions

In [ ]:
answers_ctx, refs_ctx, questions_ctx = generate_answers(
 tokenizer, model, test_dataset, True, 100)
answers_noctx, refs_noctx, questions_noctx = generate_answers(
 tokenizer, model, test_dataset, False, 100)

In [ ]:
def display_answer_and_references(question, answer, reference):
 print("Question:", question)
 print("Generated answer:", answer)
 print("Reference answer:", reference)
 print()
print("*** With context ***")
for i in range(3):
 display_answer_and_references(questions_ctx[i], answers_ctx[i],
 refs_ctx[i])
print("*** Without context ***")
for i in range(3):
 display_answer_and_references(questions_noctx[i],
 answers_noctx[i], refs_noctx[i])


In [ ]:
# Computes the average score of a given metric from a list of ROUGE scores.
def compute_average_score(scores, metric, key):
 total = 0
 for i in range(len(scores)):
  # Since it's not a map, we have to manually read the attribute.
  total += getattr(scores[i][metric], key)
 return total / len(scores)
 # Computes ROGUE-1, ROGUE-2 and ROGUE-L scores for the given generated
# answers and reference answers.
def compute_rouge(predictions, references):
 # Compute ROUGE-1, ROGUE-2 and ROUGE-L.
 metrics = ["rouge1", "rouge2", "rougeL"]

 # Use Porter stemmer to strip word suffixes to improve matching.
 scorer = rouge_scorer.RougeScorer(metrics, use_stemmer=True)
 # For each answer/reference pair, compute the ROUGE metrics.
 scores = []
 for prediction, reference in zip(predictions, references):
  scores.append(scorer.score(reference, prediction))
 # Compute the average precision, recall and F1 score for each metric.
 results = {}
 for metric in metrics:
  for k in ["precision", "recall", "fmeasure"]:
    results[f"{metric}_{k}"] = compute_average_score(
    scores, metric, k)
 return results

In [ ]:
print("ROUGE with context:", compute_rouge(answers_ctx, refs_ctx))
print()
print("ROUGE without context:", compute_rouge(answers_noctx, refs_noctx))

In [ ]:
context = """T5 is a series of large language models developed by Google AI introduced in
2019.
Like the original Transformer model, T5 models are encoder-decoder Transformers, where
the encoder processes the input text, and
the decoder generates the output text. T5 models are usually pretrained on a massive
dataset of text and code, after which they can
perform the text-based tasks that are similar to their pretrained tasks. They can also be
finetuned to perform other tasks.
T5 models have been employed in various applications, including chatbots, machine
translation systems, text summarization tools,
code generation, and robotics."""
question = "What was T5 pretrained on?"
question_and_context = encode_question_and_context(question, context)
print("Question:", question)
print("Answer:", generate_response(tokenizer, model, [question_and_context]))


In [ ]:
samples = test_dataset.select(range(5))

prompts_context = [
    encode_question_and_context(s["question"], s["context"])
    for s in samples
]

preds_context = generate_response(tokenizer, model, prompts_context)

for i, s in enumerate(samples):
    print("*With context*")
    print("Question:", s["question"])
    print("Generated answer:", preds_context[i])
    print("Reference answer:", s["answers"]["text"][0])
    print()

In [ ]:
samples = test_dataset.select(range(5))

prompts_nocontext = [f"question: {s['question']}" for s in samples]
preds_nocontext = generate_response(tokenizer, model, prompts_nocontext)

for i, s in enumerate(samples):
    print("*No context*")
    print("Question:", s["question"])
    print("Generated answer:", preds_nocontext[i])
    print("Reference answer:", s["answers"]["text"][0])
    print()

In [ ]:
model_name = "MaRiOrOsSi/t5-base-finetuned-question-answering"
tokenizer = AutoTokenizer.from_pretrained(model_name)
modefinetuned = AutoModelWithLMHead.from_pretrained(model_name)
question = "What is 42?"
context = "42 is the answer to life, the universe and everything"
input = f"question: {question} context: {context}"
encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
output = modefinetuned.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

In [ ]:
answers_ctx, refs_ctx, questions_ctx = generate_answers(
 tokenizer, modefinetuned, test_dataset, True, 100)
answers_noctx, refs_noctx, questions_noctx = generate_answers(
 tokenizer, modefinetuned, test_dataset, False, 100)

In [ ]:
print("ROUGE with context:", compute_rouge(answers_ctx, refs_ctx))
print()
print("ROUGE without context:", compute_rouge(answers_noctx, refs_noctx))

In [ ]:
samples = test_dataset.select(range(5))

prompts_context = [
    encode_question_and_context(s["question"], s["context"])
    for s in samples
]

preds_context = generate_response(tokenizer, modefinetuned, prompts_context)

for i, s in enumerate(samples):
    print("*With context*")
    print("Question:", s["question"])
    print("Generated answer:", preds_context[i])
    print("Reference answer:", s["answers"]["text"][0])
    print()

In [ ]:
samples = test_dataset.select(range(5))

prompts_nocontext = [f"question: {s['question']}" for s in samples]
preds_nocontext = generate_response(tokenizer, modefinetuned, prompts_nocontext)

for i, s in enumerate(samples):
    print("*No context*")
    print("Question:", s["question"])
    print("Generated answer:", preds_nocontext[i])
    print("Reference answer:", s["answers"]["text"][0])
    print()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "mrm8488/t5-base-finetuned-squadv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
modelsQuADv2 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

question = "What is 42?"
context = "42 is the answer to life, the universe and everything"
input = f"question: {question} context: {context}"
encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
output = modelsQuADv2.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

In [ ]:
answers_ctx, refs_ctx, questions_ctx = generate_answers(
 tokenizer, modelsQuADv2, test_dataset, True, 100)
answers_noctx, refs_noctx, questions_noctx = generate_answers(
 tokenizer, modelsQuADv2, test_dataset, False, 100)

In [ ]:
samples = test_dataset.select(range(5))

prompts_context = [
    encode_question_and_context(s["question"], s["context"])
    for s in samples
]

preds_context = generate_response(tokenizer, modelsQuADv2, prompts_context)

for i, s in enumerate(samples):
    print("*With context*")
    print("Question:", s["question"])
    print("Generated answer:", preds_context[i])
    print("Reference answer:", s["answers"]["text"][0])
    print()

In [ ]:
samples = test_dataset.select(range(5))

prompts_nocontext = [f"question: {s['question']}" for s in samples]
preds_nocontext = generate_response(tokenizer, modelsQuADv2, prompts_nocontext)

for i, s in enumerate(samples):
    print("*No context*")
    print("Question:", s["question"])
    print("Generated answer:", preds_nocontext[i])
    print("Reference answer:", s["answers"]["text"][0])
    print()